In [ ]:
#THIS PIPELINE ASSUMING YOU JUST FINISHED RUNNING d415_acquisition_color_and_rgb_and_compress.ipynb
#IF YOU ARE JUST STARTING WITH RAW BINARY FILES THEN SKIP BELOW TO WHERE IT SAYS "START HERE IF JUST WORKING WITH...
#DON'T FORGET THE IMPORTS THOUGH

In [ ]:
#DEFAULT PARAMS IN CASE YOU DON'T HAVE THE VARIABLES FILE
#IF THAT'S THE CASE YOU ALSO NEED TO SPECIFY NUMBER OF FRAMES AND BACKGROUND CHUNK LENGTH
frame_height=480
frame_width=640
fps=30
box_size=192
stringency=30
min_range=0
max_range=1000
grand_frame=0
background_chunk=600
number_of_frames=60000

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
import gzip
import matplotlib.colors as mcol
import matplotlib.cm as cm
import pickle
import os
import imageio
import gc
import cv2
from scipy import stats
from scipy import ndimage #for finding com
from scipy.misc import imsave
from itertools import cycle
cycol = cycle('bgrcmk') #here I'm generating a list of colors I can plot

In [ ]:
#Change to directory that contains the ouput of d415_acquisition_color_and_rgb_and_compress.ipynb
os.chdir("/home/rockwell/Real_Sensible_Moseq/data_acquisition/")

In [ ]:
#enter folder name
experiment=input("Regarding experiment date and number (six digit id): Enter YYMMDD_######   -")

In [ ]:
os.chdir(experiment)
os.mkdir(experiment+'_depth_frames')
os.mkdir(experiment+'_color_frames')

In [ ]:
with open(experiment+'_variables', 'rb') as variables:
    number_of_frames,frame_height,frame_width,fps,background_chunk = pickle.load(variables)
    
exp_numb=0
box_size=192
stringency=30 #10 for d415 50 for sr300
grand_frame=int(exp_numb*(number_of_frames-background_chunk))
min_range=0
max_range=300 #1000

In [ ]:
depthfilename='%s_raw_depth.gz'%experiment
colorfilename='%s_raw_color.gz'%experiment

In [ ]:
#Load and uncompress data (depth)
input_file = gzip.GzipFile(colorfilename, 'rb')
data = input_file.read()
input_file.close()

output_file = open("colordata.bin", 'wb')
output_file.write(data)
output_file.close()

In [ ]:
#Load and uncompress data (color)
input_file = gzip.GzipFile(depthfilename, 'rb')
data = input_file.read()
input_file.close()

output_file = open("depthdata.bin", 'wb')
output_file.write(data)
output_file.close()

In [ ]:
#START FROM HERE IF JUST WORKING WITH RAW BINARY FILES, YOU MAY NEED TO RENAME YOU DEPTH AND COLOR BINARY FILES TO depthdata.bin and colordata.bin

depthframes = np.fromfile('depthdata.bin', dtype=np.uint16).reshape(-1, frame_height, frame_width)[background_chunk:number_of_frames]
depth_background_frames = depthframes[0:background_chunk]
depthbackgroundmean=np.mean(depth_background_frames, axis=0) #may be better to use median
depthbackgroundvariance=np.var(depth_background_frames, axis=0)


In [ ]:
plt.imshow(depth_background_frames[6])

In [ ]:
color_background_frames = np.fromfile('colordata.bin', dtype=np.uint8, count=(int(frame_height*frame_width*background_chunk*24))).reshape(-1,frame_height, frame_width,3)[0:background_chunk]

colorbackgroundmean=np.mean(color_background_frames, axis=0) #may be better to use median
colorbackgroundvariance=np.var(color_background_frames, axis=0)

colorbackgroundmean=np.dot(colorbackgroundmean[...,:3], [.33, .34, .33])
colorbackgroundvariance=np.dot(colorbackgroundvariance[...,:3], [.33, .34, .33])

In [ ]:
plt.imshow(color_background_frames[6])

In [ ]:
min_range=0
stringency=8
current_frame=0
colorframes = np.fromfile('colordata.bin', dtype=np.uint8).reshape(-1, frame_height, frame_width,3)[background_chunk:number_of_frames]

for f in range(int(number_of_frames-background_chunk)):
        
    #FRAME TO PROCESS#################################################################
    #depth
    depth_frame_of_interest=depthframes[current_frame] 
    color_frame_of_interest=colorframes[current_frame] 
    #color
    color_frame_of_interest=np.dot(color_frame_of_interest[...,:3], [.33, .34, .33])
    
    #DEPTH BACKGROUND SUBTRACTION#####################################################
    #Background subtraction with no filters
    
    #IF PIXEL ISN'T SIGNIFICANTLY DIFFERENT FROM THE BACKGROUND THEN DROP IT
    backgroundmultiplier=np.abs(depth_frame_of_interest - depthbackgroundmean) - (stringency*np.sqrt(depthbackgroundvariance)) 
    

    #WE EXPECT POSITIVE VALUES FOR HIGH DIFFERENCES make them stay by multiplying by 1
    backgroundmultiplier[backgroundmultiplier>0]=1
    #WE EXPECT NEGATIVE VALUES FOR SMALL OR NO DIFFERENCE probably just noise, get rid of them by multiplying by 0
    backgroundmultiplier[backgroundmultiplier<=0]=0
       
    # SUBTRACT BACKGROUND BY MULTIPLYING INSIGNIFIACNT FRAMES BY ZERO
    depth_frame_of_interest_backgroundsubtracted=(np.multiply(depth_frame_of_interest,backgroundmultiplier)).astype('uint8') #IMWRITE TAKES UINT8
    depth_frame_of_interest_backgroundsubtracted[depth_frame_of_interest_backgroundsubtracted>max_range]=0
    depth_frame_of_interest_backgroundsubtracted[depth_frame_of_interest_backgroundsubtracted<=min_range]=0
        
    #FIND CENTER OF MOUSE FOR CROPPING ##########################################################################
    #ALGORITHIM FOR FINDING THE CENTER OF THE OBJECT (TO CROP THE PLOT) (this actually works better than any built in function I've tried)
    #basically, the row and column with the most non zero values should be the center
    
    #remove the triple quotes around this algorithim in order to save data as cropped pngs below
    """
    winningrow=0
    winningcountr=frame_width
    for rowpixels in range(0,frame_height):
        currentcountr=np.count_nonzero(depth_frame_of_interest_backgroundsubtracted[rowpixels,:]==0)
        if currentcountr<winningcountr:
            winningcountr=currentcountr
            winningrow=rowpixels

    winningcolumn=0
    winningcountc=frame_height
    for columnpixels in range(0,frame_width):
        currentcountc=np.count_nonzero(depth_frame_of_interest_backgroundsubtracted[:,columnpixels]==0)
        if currentcountc<winningcountc:
            winningcountc=currentcountc
            winningcolumn=columnpixels

    if winningrow<(box_size/2):
        winningrow=(int(box_size/2))
    if winningrow>(frame_height-box_size/2):
        winningrow=int(frame_height-box_size/2-1)
    if winningcolumn<(box_size/2):
        winningcolumn=(int(box_size/2))
    if winningcolumn>(frame_width-box_size/2):
        winningcolumn=int(frame_width-box_size/2-1)
    """
        
    #LABEL FRAMES ####################################################################

        #THIS IS THE FRAME NUMBER (I WILL COMBINE MANY RUNS SO I'M OUTPUTTING THE MASTER FRAME NUMBER)
    fplusd=int(f+grand_frame)
    print(fplusd)
       
    #SAVE DATA AS PNGS##################################################################
    
        # CROPPED: THIS IS HOW I SHOULD SAVE ARRAYS SO THEY COME OUT AS 192 by 192 FOR IMAGE ANALYSIS (IT'S SMALLER!)
    #imageio.imwrite(experiment + '_depth_frames/depth_frame_%06d.png' %(fplusd),depth_frame_of_interest_backgroundsubtracted[winningrow-int(box_size/2):winningrow+int(box_size/2),winningcolumn-int(box_size/2):winningcolumn+int(box_size/2)])
    #imageio.imwrite(experiment + '_color_frames/color_frame_%06d.png' %(fplusd),color_frame_of_interest[winningrow-int(box_size/2):winningrow+int(box_size/2),winningcolumn-int(box_size/2):winningcolumn+int(box_size/2)])
    
        # SAVE DATA AS PNGS (FULL SIZE)
    imageio.imwrite(experiment + '_color_frames/color_frame_%06d.png' %(fplusd),color_frame_of_interest)
    imageio.imwrite(experiment + '_color_frames/color_frame_%06d.png' %(fplusd),color_frame_of_interest)

    current_frame=current_frame+1
        

In [ ]:
plt.imshow(color_frame_of_interest)

In [ ]:
plt.imshow(np.subtract(color_frame_of_interest,(colorbackgroundmean/255)))

In [ ]:
x=np.subtract(color_frame_of_interest,(colorbackgroundmean/255))
x[x<.1]=0

In [ ]:
np.sqrt(colorbackgroundvariance)

In [ ]:
color_frame_of_interest*255

In [ ]:
np.divide(abs(np.subtract(color_frame_of_interest*255,np.sqrt(colorbackgroundvariance))),

In [ ]:
plt.imshow(x)

In [ ]:
plt.imshow(depth_frame_of_interest_backgroundsubtracted)

In [ ]:
#you can choose to remove the thing you just unzipped
#!rm colordata.bin
#!rm depthdata.bin

In [ ]:
plt.hist(frame_of_interest_backgroundsubtracted[frame_of_interest_backgroundsubtracted != 0],bins=range(-1000,1000))
plt.show()

In [ ]:
#when finished processing on the cluster I can transfer files back to my desktop with...
# !scp -r ra206@transfer.rc.hms.harvard.edu:/n/groups/datta/rockwell/Real_Sensible_Moseq/data_acquisition/180731_000002/180731_000002_depth_frames/  ~/Real_Sensible_Moseq/data

#I can make mp4s with
#!ffmpeg -r 30 -i /home/rockwell/Real_Sensible_Moseq/data_acquisition/nowalls/nowalls_depth_frame_%06d.png -c:v libx264 -vf fps=90 -pix_fmt yuv420p -r 30 nowalls_depth.mp4
